In [1]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

In [2]:
import tensorflow as tf
from typing import Any

2025-12-06 01:43:03.438122: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-06 01:43:03.480779: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-06 01:43:03.494597: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-06 01:43:03.571595: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-06 01:43:05.337956: W tensorflow/compiler/tf2

In [3]:
from mobilenetv2ssd.models.ssd.ops.postprocess_tf import *

In [4]:
from mobilenetv2ssd.core.config import load_config

In [5]:
main_cfg_path = "configs/train/default.yaml"
deploy_path = "configs/deploy/mobilenetv2_ssd_voc_jetson.yaml"

In [6]:
config = load_config(main_cfg_path,deploy_path)

In [7]:
def _read_deploy_config(config: dict[str, Any]):
    deploy_config = config['deploy']
    input_config = deploy_config['input']
    prior_config = deploy_config['priors']
    post_config = deploy_config['post_processing']
    classes_config = deploy_config['classes']
    deploy_config = {
        'input_size': {
            'image_height' : input_config.get('size',[224,224,3])[0],
            'image_width' : input_config.get('size',[224,224,3])[1],
        },
        'iou_thresh': float(post_config.get('nms_iou_threshold',0.4)),
        'variances': tf.constant(prior_config.get('variances',[0.1,0.2])),
        'max_detection': post_config.get('max_detections',100),
        'per_class_top_k': post_config.get('per_class_top_k',100),
        'score_thresh': post_config.get('score_threshold',0.45),
        'num_classes': classes_config.get('num_classes',21),
        'labels_map' : classes_config.get('labels_map',None),
        'use_sigmoid': classes_config.get('use_sigmoid',False)
    }
    return deploy_config

In [8]:
deploy_config = _read_deploy_config(config)

I0000 00:00:1765003387.628499   88048 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1765003387.963410   88048 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1765003387.963553   88048 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1765003387.966944   88048 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1765003387.967020   88048 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [9]:
deploy_config

{'input_size': {'image_height': 300, 'image_width': 300},
 'iou_thresh': 0.5,
 'variances': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.1, 0.2], dtype=float32)>,
 'max_detection': 200,
 'per_class_top_k': 100,
 'score_thresh': 0.3,
 'num_classes': 3,
 'labels_map': 'src/notebooks/voc_classes.txt',
 'use_sigmoid': False}

In [22]:
from pathlib import Path
def _load_label_map(label_file_path: str, use_sigmoid: bool = False):
    
    label_file_path = Path(label_file_path)
    
    with open(label_file_path, "r") as f:
        labels = [line.strip() for line in f.readlines() if line.strip()]

    # Build lookup
    if use_sigmoid:
        label_dict = {idx: name for idx, name in enumerate(labels)}
    else:
        # Softmax so labels begin at 1
        label_dict = {idx + 1: name for idx, name in enumerate(labels)}
        label_dict[0] = "background"
        
    return label_dict

In [24]:
def _decode_class_names(class_id_tensor: tf.Tensor, labels: list[str]):
    class_ids = class_id_tensor.numpy()
    decoded = [[labels[i] for i in row] for row in class_ids]
    return decoded

In [25]:
_load_label_map('./voc_labels.txt',use_sigmoid = True)

{0: 'aeroplane',
 1: 'bicycle',
 2: 'bird',
 3: 'boat',
 4: 'bottle',
 5: 'bus',
 6: 'car',
 7: 'cat',
 8: 'chair',
 9: 'cow',
 10: 'diningtable',
 11: 'dog',
 12: 'horse',
 13: 'motorbike',
 14: 'person',
 15: 'pottedplant',
 16: 'sheep',
 17: 'sofa',
 18: 'train',
 19: 'tvmonitor'}

In [26]:
def build_decoded_boxes(config: dict[str,Any], predicted_offsets: tf.Tensor, predicted_logits: tf.Tensor, priors: tf.Tensor):
    deploy_config = _read_deploy_config(config)

    # Decoding boxes
    nmsed_boxes,nmsed_scores, nmsed_classes, valid_detections = decode_and_nms(predicted_offsets = predicted_offsets, predicted_logits = predicted_logits, priors = priors, variances = deploy_config['variances'],scores_thresh = deploy_config['score_thresh'], iou_thresh = deploy_config['iou_thresh'], top_k = deploy_config['per_class_top_k'], max_detections = deploy_config['max_detection'],image_meta = deploy_config['input_size'],use_sigmoid = deploy_config['use_sigmoid'])   

    # Getting the classes
    classes = _load_label_map(Path('./voc_labels.txt'),use_sigmoid = deploy_config['use_sigmoid'])

    decoded_classes = _decode_class_names(nmsed_classes, classes)

    return nmsed_boxes, nmsed_scores, nmsed_classes, decoded_classes

In [27]:
pred_loc = tf.constant(
    [
  [  # batch 0
    [ 0.0,  0.0,  0.0,  0.0],   # anchor 0
    [ 0.2,  0.0,  0.0,  0.0],   # anchor 1
    [ 0.0,  0.0,  0.5,  0.0],   # anchor 2
    [ 0.0, -0.2,  0.0, -0.5],   # anchor 3
  ]
]
)

pred_logits = tf.constant(
    [
  [  # batch 0
    [0.1,  2.0,  0.0],   # anchor 0
    [0.0,  0.5,  3.0],   # anchor 1
    [0.5,  1.5, -1.0],   # anchor 2
    [0.2, -0.5,  0.0],   # anchor 3
  ]
]
)

priors = tf.constant(
    [
  [0.25, 0.25, 0.2, 0.2],  # anchor 0 (top-left)
  [0.75, 0.25, 0.2, 0.2],  # anchor 1 (top-right)
  [0.25, 0.75, 0.2, 0.2],  # anchor 2 (bottom-left)
  [0.75, 0.75, 0.2, 0.2],  # anchor 3 (bottom-right)
]
)

In [28]:
build_decoded_boxes(config,pred_loc,pred_logits, priors)

(<tf.Tensor: shape=(1, 200, 4), dtype=float32, numpy=
 array([[[ 45.     , 196.2    , 105.     , 256.2    ],
         [ 45.     ,  45.     , 105.     , 105.     ],
         [195.     ,  41.84487, 255.     , 108.15513],
         [196.65488, 195.     , 250.94511, 255.     ],
         [  0.     ,   0.     ,   0.     ,   0.     ],
         [  0.     ,   0.     ,   0.     ,   0.     ],
         [  0.     ,   0.     ,   0.     ,   0.     ],
         [  0.     ,   0.     ,   0.     ,   0.     ],
         [  0.     ,   0.     ,   0.     ,   0.     ],
         [  0.     ,   0.     ,   0.     ,   0.     ],
         [  0.     ,   0.     ,   0.     ,   0.     ],
         [  0.     ,   0.     ,   0.     ,   0.     ],
         [  0.     ,   0.     ,   0.     ,   0.     ],
         [  0.     ,   0.     ,   0.     ,   0.     ],
         [  0.     ,   0.     ,   0.     ,   0.     ],
         [  0.     ,   0.     ,   0.     ,   0.     ],
         [  0.     ,   0.     ,   0.     ,   0.     ],
         [ 